# Forming Queries

Get Familiar with querying the database. BUT don't forget your [cheat sheets](https://snowexsql.readthedocs.io/en/latest/cheat_sheet.html)! 

## Process
### Getting Connected
Getting connected to the database is easiest done using the snowexsql library function [`get_db`](https://snowexsql.readthedocs.io/en/latest/snowexsql.html#snowexsql.db.get_db)


In [1]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'


### Importing the tables classes
These are critical for build queries. You will need at least one of these every query since they reflect the data were interested in.


In [2]:
from snowexsql.data import SiteData, PointData, LayerData, ImageData

### Query Time!
We build queries in python using `session.query()`. Whatever we put inside of the query parentheses is what we will get back in the results!

In [7]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Lets grab a single row from the points table
qry = session.query(PointData).limit(1) # Not super useful for science, but useful for checking to be sure you don't download too much data! 
# Returns one single point 

# Execute that query!
result = qry.all()

session.close()

Pause for moment and consider what is in `result`....


Is it:

    A. a single value
    B. a bunch of values
    C. an object
    D. a row of values
 

In [9]:
# uncomment the line below and print out the results 
print(result)
# how to get at the value for this data? 
# got back a single row in the points data table... any column in this row use .value(?) 

This feels soooo *limited* :)

**Questions**
* What happens if we changed the number in the limit? What will we get back?
* Where are our column names?
* What if I only wanted a single column and not a whole row?


## Filtering
The database had a silly number of records, and asking for all of them will crash your computer. 

So let talk about using `.filter()`

All queries can be reduced by applying `session.query(__).filter(__)` and a lot can go into the parentheses. This is where your cheat sheet will come in handy.

In [15]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name) # try to bracket this with session.close() as closely as possible as not to be kicked from the database 

# Its convenient to store a query like the following 
qry = session.query(LayerData.value, LayerData.site_id) # without .value or others returns object, by specifying multiple returns a tuple

# Then filter on it to just density profiles
qry = qry.filter(LayerData.type == 'density') # pull density profile 
qry = qry.filter(LayerData.site_name == 'Grand Mesa') # pull data from location 

# protect ourselves from a lot of data
qry = qry.limit(5) # only pull 5 points 

result = qry.all()
print(result)

session.close()

[('192.0', '6C10'), ('181.0', '6C10'), ('209.5', '6C10'), ('281.5', '6C10'), ('241.0', '6C10')]


## Attempted this for plotting data from two sites instead of one
mica prime gists on github to find solution to later part 7 bonus challenge 

In [26]:
# Attempted this to plot data on two sites instead of one... 
# Does not work at the moment 

# This is what you will use for all of hackweek to access the db
from snowexsql.conversions import query_to_geopandas
engine, session = get_db(db_name) # try to bracket this with session.close() as closely as possible as not to be kicked from the database 

# Its convenient to store a query like the following 
qry = session.query(LayerData.value, LayerData.site_id) # without .value or others returns object, by specifying multiple returns a tuple

# Then filter on it to just density profiles
qry = qry.filter(LayerData.type == 'density') # pull density profile 
qry = qry.filter(LayerData.site_name == 'Grand Mesa') # pull data from location 
qry = qry.filter(LayerData.site_id.in_(['1C14', '1C1']))

# protect ourselves from a lot of data
qry = qry.limit(2) # only pull 5 points 

result = query_to_geopandas(qry, engine) 
print(result)
reult.plot()

session.close()

ValueError: Query missing geometry column 'geom'

**Questions**
* What happens if I filter on a qry that's been filtered?
* What happens if I just want a single column/attribute back? How do I do that?

### How do I know what to filter on?
Queries and `.distinct()`!

In [22]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Get the unique datanames in the table
results = session.query(PointData.instrument).distinct().all()
#print('Available Instruments = {}'.format(', '.join([r[0] for r in results])))
print('Available Instruments = {}'.format(', '.join([str(r[0]) for r in results])))


# Get the unique instrument in the table
#results = session.query(LayerData.instrument).distinct().all()
#print('\nAvailable Instruments = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique dates in the table
results = session.query(PointData.date).distinct().all()
print('\nAvailable Dates = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique surveyors in the table
#results = session.query(LayerData.observers).distinct().all()
#print('\nAvailable surveyors = {}'.format(', '.join([str(r[0]) for r in results])))

session.close()

Available Instruments = None, Mala 1600 MHz GPR, Mala 800 MHz GPR, pulse EKKO Pro multi-polarization 1 GHz GPR, pit ruler, mesa, magnaprobe, camera

Available Dates = 2020-05-28, 2020-01-09, 2020-05-23, 2019-11-29, 2020-01-04, 2019-10-20, 2019-11-30, 2020-04-17, 2020-02-19, 2020-02-26, 2020-02-03, 2020-05-05, 2019-10-05, 2019-12-29, 2020-06-02, 2019-10-28, 2020-01-30, 2020-05-22, 2020-03-09, 2019-12-09, 2019-12-28, 2020-02-24, 2020-03-17, 2020-04-01, 2020-05-14, 2019-10-14, 2019-10-29, 2019-10-02, 2020-01-31, 2020-04-18, 2020-04-26, 2019-10-12, 2020-04-29, 2020-02-23, 2020-01-22, 2020-01-01, 2019-11-21, 2020-05-10, 2020-02-12, 2019-11-19, 2020-05-06, 2019-10-25, 2019-11-02, 2020-02-08, 2020-04-14, 2020-04-02, 2019-11-16, 2020-04-07, 2019-12-27, 2019-10-01, 2020-04-16, 2020-06-08, 2019-12-13, 2019-10-17, 2019-10-22, 2020-04-21, 2020-01-03, 2019-12-12, 2019-12-08, 2020-01-25, 2020-02-29, 2019-11-24, 2019-10-18, 2020-05-09, 2020-03-22, 2019-11-06, 2019-12-16, 2020-01-15, 2019-11-22, 2019-

## Recap 
You just explored using the session object to form queries and compounding filters results with it

**You should know:**
* How to build queries using filtering
* How to isolate column data 
* Determine what values to filter on

If you don't feel comfortable with these, you are probably not alone, let's discuss it!